In [1]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()
# df = pd.DataFrame(boston.data.T, ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']) #有13個feature
df = pd.DataFrame(boston.data.T, ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']).T #有13個feature
df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


資料說明
* CRIM：按城鎮劃分的人均犯罪率
* ZN：超過25,000平方英尺的土地劃為住宅用地的比例
* INDUS：城鎮非零售商用土地的比例
* CHAS：Charles River虛擬變數(如果靠近河流為1；否則為0)
* NOX：一氧化氮濃度(以百萬分之幾為單位)
* RM：每個住宅的平均房間數
* AGE：1940年之前建造自有單位的比例
* DIS：到五個波士頓就業中心的加權距離
* RAD：徑向公路的可達性指標
* TAX：每10,000美金的全值財產稅率
* PTRATIO：城鎮的師生比例
* B：1000(Bk-0.63)^2，其中Bk是按城鎮劃分的非裔比例
* LSTAT：低階人口狀況百分比
* MEDV：自有住房的中位數價格(單位為1,000美元)

In [17]:
X = pd.DataFrame(boston.data)
y = pd.DataFrame(boston.target)
# y = np.squeeze(y)

print(X.shape)
print(y.shape)

(506, 13)
(506, 1)


In [18]:
# TODO
#分出20%的資料作為test set，random_state = 1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(404, 13) (102, 13) (404, 1) (102, 1)


In [19]:
# TODO
#Fit linear model 配適線性模型
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [20]:
# TODO
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
y_pred = regressor.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred, squared=True))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('R2:', r2_score(y_test, y_pred))
print('Score:',regressor.score(X_test, y_test))#前者代表我放進去的input，後者代表"解答"。我們需要將input丟入方程式後得到的結果去跟解答做相除來得到分數

MAE: 3.7507121808389092
MSE: 23.38083648026999
RMSE: 4.835373458200514
R2: 0.7634174432138495
Score: 0.7634174432138495


請問，預測房價為何？

In [21]:
# TODO
X_new = ([[0.00632, 18.00, 2.310, 0, 0.5380, 6.5750, 65.20, 4.0900, 1, 296.0, 15.30, 396.90 , 4.98]])
y_new = regressor.predict(X_new)
print(y_new)

[[30.06627665]]


請問，哪些特徵在反向淘汰下，可以被移除？(P-value是否小於0.05)

In [22]:
# TODO
import statsmodels.api as sm
X_train_c = sm.add_constant(X_train)
X_train_c = np.array(X_train_c, dtype='float')
X_opt = X_train_c[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]

regressor_OLS = sm.OLS(y_train, X_opt).fit()
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      0   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     80.85
Date:                Tue, 08 Mar 2022   Prob (F-statistic):          5.53e-102
Time:                        08:03:33   Log-Likelihood:                -1196.4
No. Observations:                 404   AIC:                             2421.
Df Residuals:                     390   BIC:                             2477.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         42.9335      5.752      7.464      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [23]:
# TODO
X_opt = X_train_c[:, [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13]]

regressor_OLS = sm.OLS(y_train, X_opt).fit()
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      0   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     95.98
Date:                Tue, 08 Mar 2022   Prob (F-statistic):          6.22e-104
Time:                        08:03:35   Log-Likelihood:                -1196.5
No. Observations:                 404   AIC:                             2417.
Df Residuals:                     392   BIC:                             2465.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         42.6921      5.706      7.481      0.0